In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [10]:
def parse_minutes(x):
    
    spl = x.split('.')[0].split(' ')
    hours = int(spl[0]) * 60
    mins = int(spl[2]) if spl[2] != '' else 0
    return hours + mins

def read_map(plate_map, ctrl='H', num_inducers=1, induction_lvl=12, ctrl_replicate=2):
    
    plate_map = plate_map.melt(id_vars=['group'])
    plate_map['variable'] = plate_map['variable'].astype(int)
    plate_map['Well'] = plate_map['group'] + plate_map['variable'].apply(lambda x: '{:02d}'.format(x))
    plate_map.loc[~plate_map['group'].isin([ctrl]), 'suffix'] = (plate_map['variable'] - 1) % induction_lvl
    plate_map.loc[plate_map['group'].isin([ctrl]), 'suffix'] = (plate_map['variable'] - 1) % ctrl_replicate
    return plate_map[['Well', 'value', 'suffix']].dropna()

def generate_metadata(well, plate_map):
    
    dictionary = pd.read_csv('datasets/dictionary.csv')
    sample_map = {
        'P62': 'pBAD-rJ/Ara',
        'P63': 'araC-pBAD-rJ/Ara',
        'P64': 'pCin-rJ/OHC14',
        'P65': 'pCymRC-rJ/Cuma',
        'P66': 'cymRC-pCymRC-rJ/Cuma',
        'P67': 'pLux2-rJ/AHL',
        'P68': 'pLuxB-rJ/AHL',
        'P69': 'pPhlF-rJ/DAPG',
        'P70': 'pSalTTC-rJ/Sal',
        'P71': 'pVanCC-rJ/Van',
        'P72': 'rhaS-pRhaB-rJ/Rha',
        'P73': 'araC-pBAD/Ara',
        'E720': 'e11x33',
        'E721': 'e15x33',
        'E722': 'e16x33',
        'E723': 'e20x33',
        'E724': 'e32x33',
        'E725': 'e34x33',
        'E726': 'e38x33',
        'E727': 'e41x33',
        'E728': 'e42x33',
        'G726': 'e11x32STPhoRadA',
        'G721': 'e15x32NpuSspS2',
        'G722': 'e16x33NrdA2',
        'G723': 'e20x32gp411',
        'G724': 'e32x30SspGyrB',
        'G725': 'e34x30MjaKlbA',
        'G726': 'e38x32gp418',
        'G727': 'e41x32NrdJ1',
        'G728': 'e42x32STIMPDH1',
        'BK': 'blank-kan',
        'BA': 'blank-amp',
        '3K3-N': 'negative-control-kan',
        '4A3-N': 'negative-control-amp',
        '4A3-P': 'positive-control-amp'
    }
    #sample_map.update(control_map)
    df = pd.merge(well, plate_map, on='Well', how='left').dropna(subset=['value']).reset_index(drop=True)
    df['short_name'] = df['value'].map(sample_map) + '_' + df['suffix'].astype(int).astype(str)
    return df.dropna()


def transpose_data(df, col):
    
    df.set_index(col, inplace=True)
    df = df.transpose().reset_index()
    df['time'] = df['index'].apply(parse_minutes)
    df = df.set_index('time').drop('index', axis=1)
    return df

def generate_data(df, name, h=24, m=0, num_data=4, start_idx=3, col='short_name'):
    
    datas = []
    idxs = [start_idx]
    for i in range(1, num_data+1):
        idxs.append(h * 3 + idxs[i-1] + 1 + int(m/20))
        data = (df.iloc[:, idxs[i-1]:idxs[i]]).astype(float)
        data = pd.concat([name, data], axis=1)
        data = transpose_data(data, col)
        datas.append(data)
    return datas

In [11]:
folder = '029-new-promoters-characterization'
plate_map = read_map(pd.read_csv('datasets/experiment/{}/plate_map.csv'.format(folder)))
raw_data = pd.read_csv('datasets/experiment/{}/raw.csv'.format(folder), skiprows=[0])
metadata = generate_metadata(raw_data['Well'], plate_map)
data = raw_data[raw_data['Well'].isin(metadata['Well'])].reset_index(drop=True)
datas = generate_data(data, metadata['short_name'], h=22, m=20, num_data=8)

In [12]:
datas[4]

short_name,pBAD-rJ/Ara_0,pBAD-rJ/Ara_1,pBAD-rJ/Ara_2,pBAD-rJ/Ara_3,pBAD-rJ/Ara_4,pBAD-rJ/Ara_5,pBAD-rJ/Ara_6,pBAD-rJ/Ara_7,pBAD-rJ/Ara_8,pBAD-rJ/Ara_9,pBAD-rJ/Ara_10,pBAD-rJ/Ara_11,araC-pBAD-rJ/Ara_0,araC-pBAD-rJ/Ara_1,araC-pBAD-rJ/Ara_2,araC-pBAD-rJ/Ara_3,araC-pBAD-rJ/Ara_4,araC-pBAD-rJ/Ara_5,araC-pBAD-rJ/Ara_6,araC-pBAD-rJ/Ara_7,araC-pBAD-rJ/Ara_8,araC-pBAD-rJ/Ara_9,araC-pBAD-rJ/Ara_10,araC-pBAD-rJ/Ara_11,cymRC-pCymRC-rJ/Cuma_0,cymRC-pCymRC-rJ/Cuma_1,cymRC-pCymRC-rJ/Cuma_2,cymRC-pCymRC-rJ/Cuma_3,cymRC-pCymRC-rJ/Cuma_4,cymRC-pCymRC-rJ/Cuma_5,cymRC-pCymRC-rJ/Cuma_6,cymRC-pCymRC-rJ/Cuma_7,cymRC-pCymRC-rJ/Cuma_8,cymRC-pCymRC-rJ/Cuma_9,cymRC-pCymRC-rJ/Cuma_10,cymRC-pCymRC-rJ/Cuma_11,pLux2-rJ/AHL_0,pLux2-rJ/AHL_1,pLux2-rJ/AHL_2,pLux2-rJ/AHL_3,pLux2-rJ/AHL_4,pLux2-rJ/AHL_5,pLux2-rJ/AHL_6,pLux2-rJ/AHL_7,pLux2-rJ/AHL_8,pLux2-rJ/AHL_9,pLux2-rJ/AHL_10,pLux2-rJ/AHL_11,pPhlF-rJ/DAPG_0,pPhlF-rJ/DAPG_1,pPhlF-rJ/DAPG_2,pPhlF-rJ/DAPG_3,pPhlF-rJ/DAPG_4,pPhlF-rJ/DAPG_5,pPhlF-rJ/DAPG_6,pPhlF-rJ/DAPG_7,pPhlF-rJ/DAPG_8,pPhlF-rJ/DAPG_9,pPhlF-rJ/DAPG_10,pPhlF-rJ/DAPG_11,rhaS-pRhaB-rJ/Rha_0,rhaS-pRhaB-rJ/Rha_1,rhaS-pRhaB-rJ/Rha_2,rhaS-pRhaB-rJ/Rha_3,rhaS-pRhaB-rJ/Rha_4,rhaS-pRhaB-rJ/Rha_5,rhaS-pRhaB-rJ/Rha_6,rhaS-pRhaB-rJ/Rha_7,rhaS-pRhaB-rJ/Rha_8,rhaS-pRhaB-rJ/Rha_9,rhaS-pRhaB-rJ/Rha_10,rhaS-pRhaB-rJ/Rha_11,araC-pBAD/Ara_0,araC-pBAD/Ara_1,araC-pBAD/Ara_2,araC-pBAD/Ara_3,araC-pBAD/Ara_4,araC-pBAD/Ara_5,araC-pBAD/Ara_6,araC-pBAD/Ara_7,araC-pBAD/Ara_8,araC-pBAD/Ara_9,araC-pBAD/Ara_10,araC-pBAD/Ara_11,blank-kan_0,blank-kan_1,negative-control-kan_0,negative-control-kan_1,blank-amp_0,blank-amp_1,negative-control-amp_0,negative-control-amp_1,positive-control-amp_0,positive-control-amp_1
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.162,0.201,0.156,0.159,0.142,0.125,0.118,0.125,0.164,0.120,0.139,0.143,0.120,0.160,0.122,0.145,0.157,0.123,0.120,0.121,0.121,0.126,0.129,0.130,0.191,0.166,0.153,0.153,0.118,0.150,0.125,0.156,0.128,0.118,0.122,0.139,0.155,0.128,0.130,0.218,0.130,0.125,0.120,0.146,0.120,0.144,0.140,0.134,0.148,0.135,0.135,0.165,0.158,0.138,0.138,0.148,0.144,0.136,0.142,0.132,0.127,0.135,0.129,0.122,0.124,0.123,0.158,0.125,0.122,0.124,0.122,0.127,0.136,0.118,0.949,0.123,0.119,0.119,0.132,0.123,0.122,0.138,0.127,0.129,0.121,0.121,0.338,0.210,0.121,0.121,0.128,0.127,0.154,0.145
20,0.141,0.159,0.139,0.141,0.133,0.132,0.134,0.126,0.132,0.132,0.141,0.138,0.133,0.137,0.135,0.134,0.133,0.129,0.133,0.128,0.134,0.130,0.134,0.135,0.130,0.133,0.132,0.133,0.132,0.138,0.130,0.135,0.132,0.132,0.131,0.138,0.139,0.133,0.133,0.137,0.135,0.135,0.134,0.132,0.135,0.132,0.131,0.134,0.145,0.135,0.135,0.139,0.136,0.139,0.136,0.138,0.135,0.134,0.135,0.138,0.137,0.133,0.130,0.133,0.133,0.135,0.134,0.132,0.135,0.133,0.130,0.143,0.140,0.116,0.130,0.137,0.134,0.135,0.137,0.133,0.135,0.132,0.179,0.137,0.119,0.120,0.218,0.194,0.119,0.120,0.133,0.138,0.133,0.139
40,0.145,0.161,0.143,0.143,0.134,0.134,0.135,0.128,0.134,0.132,0.143,0.140,0.134,0.139,0.136,0.135,0.135,0.131,0.134,0.130,0.136,0.132,0.136,0.138,0.133,0.134,0.134,0.133,0.133,0.140,0.133,0.136,0.133,0.133,0.134,0.140,0.145,0.137,0.136,0.140,0.138,0.137,0.136,0.133,0.136,0.135,0.133,0.137,0.150,0.139,0.138,0.144,0.138,0.142,0.139,0.139,0.138,0.135,0.136,0.140,0.139,0.137,0.132,0.135,0.135,0.135,0.135,0.135,0.136,0.133,0.132,0.143,0.146,0.116,0.132,0.142,0.136,0.136,0.138,0.134,0.135,0.133,0.134,0.139,0.119,0.119,0.245,0.209,0.121,0.121,0.138,0.139,0.133,0.140
60,0.163,0.174,0.157,0.156,0.141,0.140,0.144,0.134,0.141,0.139,0.150,0.147,0.142,0.150,0.144,0.145,0.141,0.138,0.142,0.137,0.144,0.138,0.143,0.143,0.142,0.142,0.139,0.139,0.139,0.149,0.140,0.143,0.139,0.138,0.138,0.145,0.156,0.145,0.144,0.151,0.146,0.144,0.143,0.138,0.142,0.138,0.140,0.143,0.157,0.147,0.145,0.152,0.145,0.150,0.143,0.144,0.143,0.140,0.142,0.144,0.147,0.143,0.138,0.140,0.139,0.141,0.140,0.138,0.141,0.137,0.136,0.148,0.155,0.117,0.138,0.147,0.140,0.142,0.144,0.139,0.140,0.138,0.138,0.143,0.119,0

In [13]:
datas[0].to_csv('datasets/experiment/{}/ods.csv'.format(folder))
datas[1].to_csv('datasets/experiment/{}/fluos.csv'.format(folder))
datas[2].to_csv('datasets/experiment/{}/fluos-lower-1.csv'.format(folder))
datas[3].to_csv('datasets/experiment/{}/fluos-lower-2.csv'.format(folder))
datas[4].to_csv('datasets/experiment/{}/raw-ods.csv'.format(folder))
datas[5].to_csv('datasets/experiment/{}/raw-fluos.csv'.format(folder))
datas[6].to_csv('datasets/experiment/{}/raw-fluos-lower-1.csv'.format(folder))
datas[7].to_csv('datasets/experiment/{}/raw-fluos-lower-2.csv'.format(folder))